# About

This notebook provides example code for connecting to and interacting with the Watson Tone Analyzer service.

## What You Need
The username and password of a Watson account, set either directly or using a separate file. An example json file is provided which you will need to adjust before running the below code.

If set directly, do not push credentials in notebook into Git.

## Imports

In [ ]:
# General
import datetime
import json

# Data structures
import pandas as pd

# Api connection imports
from requests_oauthlib import OAuth1
from watson_developer_cloud import ToneAnalyzerV3

## Constants

In [ ]:
# Possible tones that the api will classify against
LIST_OF_TONES = ['Analytical', 'Tentative', 'Joy', 'Sadness', 'Anger', 'Confident', 'Fear']

CONTENT_TYPE = 'application/json'
TONE_ANALYZER_URL = "https://gateway.watsonplatform.net/tone-analyzer/api"

# You can replace this with current date to get the most current version
# Different versions may be different, so when in production, use the same version for everything
VERSION = '2018-05-10'

## API Connection

In [ ]:
with open('creds_example.json') as f:
    creds = json.load(f)

# Create Tone Analyzer Connection
tone_analyzer = ToneAnalyzerV3(
    username=creds['tone_analyzer']['username'],
    password=creds['tone_analyzer']['password'],
    version=VERSION,
    url=TONE_ANALYZER_URL
)

# Get Tones

In [ ]:
# Returns all information json datatype
# Will include sentence by sentence tone if available
# Use this if you want to explore everything returned by the call
def get_raw_tone_json(text):
    tone = tone_analyzer.tone({"text": text}, content_type)
    return tone


# Takes text, returns a dictionary of overall document tones and their scores from Watson
# If it fails, returns a dictionary of None objects
# Can fail for many reasons, including just timing out, so error handling should be more robust than this
# Does not return sentence by sentence tones
# TODO: better error handling
def get_clean_tones(text):
    dict_of_tones = dict()
    try:
        tone = tone_analyzer.tone({"text": text}, content_type)
        for tone_results in tone['document_tone']['tones']:
            dict_of_tones[tone_results['tone_name']] = tone_results['score']
        return dict_of_tones
    except:
        for tone in LIST_OF_TONES:
            dict_of_tones[tone] = None
        return dict_of_tones